In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi = init_hmm(T=50)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.partial(combinators.sequence, hmm_step)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi = hmm_run(50, z0, mu, sigma, pi)

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.zeros(5),
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [12]:
smc_hmm = smc.smc(hmm.hmm_step, hmm.hmm_retrace, reparameterized=False)

In [13]:
inference_init = combinators.Model(hmm.init_hmm, smc_hmm_params, {})

inference = smc.ParticleTrace(num_particles)
inference_init.condition(trace=inference, observations=generative)
smc_hmm.condition(trace=inference, observations=generative)

In [14]:
inference, smc_hmm_params = smc.variational_smc(num_particles, hmm.init_hmm, smc_hmm, 500, 50, smc_hmm_params, generative, use_cuda=False, marginal_model='hmm_step', lr=1e-2)

07/11/2018 15:07:51 Variational SMC ELBO=-1.63285217e+02 at epoch 1
07/11/2018 15:07:52 Variational SMC ELBO=-1.72401154e+02 at epoch 2
07/11/2018 15:07:52 Variational SMC ELBO=-1.68506714e+02 at epoch 3
07/11/2018 15:07:52 Variational SMC ELBO=-1.69398163e+02 at epoch 4
07/11/2018 15:07:53 Variational SMC ELBO=-1.60190079e+02 at epoch 5
07/11/2018 15:07:53 Variational SMC ELBO=-1.64842422e+02 at epoch 6
07/11/2018 15:07:53 Variational SMC ELBO=-1.62738358e+02 at epoch 7
07/11/2018 15:07:54 Variational SMC ELBO=-1.61993423e+02 at epoch 8
07/11/2018 15:07:54 Variational SMC ELBO=-1.55281296e+02 at epoch 9
07/11/2018 15:07:54 Variational SMC ELBO=-1.60252655e+02 at epoch 10
07/11/2018 15:07:54 Variational SMC ELBO=-1.73509216e+02 at epoch 11
07/11/2018 15:07:55 Variational SMC ELBO=-1.62971008e+02 at epoch 12
07/11/2018 15:07:55 Variational SMC ELBO=-1.70499573e+02 at epoch 13
07/11/2018 15:07:55 Variational SMC ELBO=-1.60576324e+02 at epoch 14
07/11/2018 15:07:56 Variational SMC ELBO=-1

In [15]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum() / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.000000
SMC accuracy at time 2: 0.000000
SMC accuracy at time 3: 0.000000
SMC accuracy at time 4: 0.000000
SMC accuracy at time 5: 0.000000
SMC accuracy at time 6: 0.000000
SMC accuracy at time 7: 0.000000
SMC accuracy at time 8: 0.000000
SMC accuracy at time 9: 0.000000
SMC accuracy at time 10: 0.000000
SMC accuracy at time 11: 0.000000
SMC accuracy at time 12: 0.000000
SMC accuracy at time 13: 0.000000
SMC accuracy at time 14: 0.000000
SMC accuracy at time 15: 0.000000
SMC accuracy at time 16: 0.000000
SMC accuracy at time 17: 0.000000
SMC accuracy at time 18: 0.000000
SMC accuracy at time 19: 0.000000
SMC accuracy at time 20: 0.000000
SMC accuracy at time 21: 0.000000
SMC accuracy at time 22: 0.000000
SMC accuracy at time 23: 0.000000
SMC accuracy at time 24: 0.000000
SMC accuracy at time 25: 0.000000
SMC accuracy at time 26: 0.000000
SMC accuracy at time 27: 0.000000
SMC accuracy at time 28: 0.000000
SMC accuracy at time 29: 0.000000
SMC accuracy at time 30

In [16]:
smc_hmm_params

"{'Pi_0__concentration': 'Parameter containing:\ntensor([ 1.4738,  1.4358,  1.5176,  1.2865,  1.3441])', 'Pi_1__concentration': 'Parameter containing:\ntensor([ 0.6017,  1.5267,  1.0329,  1.1116,  1.2111])', 'Pi_2__concentration': 'Parameter containing:\ntensor([ 1.1566,  1.4418,  1.2621,  1.2510,  1.2458])', 'Pi_3__concentration': 'Parameter containing:\ntensor([ 0.9080,  1.6801,  0.7114,  1.1705,  1.2632])', 'Pi_4__concentration': 'Parameter containing:\ntensor([ 0.9604,  1.5793,  1.0788,  0.7007,  1.1923])', 'Pi_5__concentration': 'Parameter containing:\ntensor([ 1.1125,  1.6441,  1.2444,  1.0540,  0.8094])', 'mu__loc': 'Parameter containing:\ntensor([ 1.6369,  2.6796,  1.9857,  1.9331,  1.8844])', 'mu__scale': 'Parameter containing:\ntensor([ 1.3058,  2.6194,  1.7108,  1.5725,  1.8202])', 'sigma__loc': 'Parameter containing:\ntensor([ 2.0942,  3.1583,  2.5592,  2.5695,  2.5155])', 'sigma__scale': 'Parameter containing:\ntensor([ 1.5918,  1.2851,  1.5448,  1.5645,  1.4387])'}"